In [3]:
import sys
print sys.executable

paths = ['', '/Users/dyawitz/anaconda/bin', '/Users/dyawitz/anaconda/lib/python27.zip', '/Users/dyawitz/anaconda/lib/python2.7/plat-darwin', '/Users/dyawitz/anaconda/lib/python2.7/plat-mac', '/Users/dyawitz/anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/dyawitz/anaconda/lib/python2.7/lib-tk', '/Users/dyawitz/anaconda/lib/python2.7/lib-old', '/Users/dyawitz/anaconda/lib/python2.7/lib-dynload', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/setuptools-18.5-py2.7.egg', '/Users/dyawitz/anaconda/lib/python2.7/site-packages', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/aeosa', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/IPython/extensions', '/Users/dyawitz/.ipython']

for i in paths:
   sys.path.append(i)

/Users/dyawitz/anaconda/envs/py27/bin/python


In [66]:
import pandas as pd
import numpy as np
import time
import random
import joblib
import os

In [5]:
weather_dict = joblib.load('weather_dict.pkl')

In [6]:
weather_dict

{'CAPITL': ('kalb', 'Capital', 'Albany'),
 'CENTRL': ('ksyr', 'Central', 'Syracuse'),
 'DUNWOD': ('klga', 'Dunwoodie', 'Yonkers'),
 'GENESE': ('kroc', 'Genese', 'Rochester'),
 'HUD VL': ('kpou', 'Hudson Valley', 'Poughkeepsie'),
 'LONGIL': ('kjfk', 'Long Island', 'NYC'),
 'MHK VL': ('krme', 'Mohawk Valley', 'Utica'),
 'MILLWD': ('klga', 'Millwood', 'Yonkers'),
 'N.Y.C.': ('kjfk', 'NYC', 'NYC'),
 'NORTH': ('kpbg', 'North', 'Plattsburgh'),
 'WEST': ('kbuf', 'West', 'Buffalo')}

In [8]:
airports = [i[0] for i in weather_dict.values()]

In [13]:
#remove duplicates
airports = list(set(airports))

In [15]:
airports.pop(0)

'kalb'

In [16]:
airports

['kpou', 'kroc', 'kpbg', 'kbuf', 'kjfk', 'ksyr', 'krme', 'klga']

In [89]:
dates = pd.date_range(pd.to_datetime('2001-05-01'), \
                       pd.to_datetime('2016-03-11'), freq='D')

In [83]:
def write_daily_weather_data(airport, dates):
    ''''''
    for d in dates:

        df0 = pd.read_csv('https://www.wunderground.com/history/airport/{0}/{1}/{2}/{3}/DailyHistory.html?format=1'\
                             .format(airport, d.year, d.month, d.day))
        cols = df0.columns
        df0.columns = [col.lower().replace(' ','').replace('<br/>', '') for col in cols]

        df0.dateutc = df0.dateutc.apply(lambda x: pd.to_datetime(x.replace('<br />', '')))

        df0.gustspeedmph = df0.gustspeedmph.replace('-', 0)
        df0.windspeedmph = df0.windspeedmph.replace('Calm', 0)
        df0.precipitationin = df0.precipitationin.replace('NaN', 0)
        df0.events = df0.events.replace('NaN', 0)

        filepath = '../data/wunderground/'+ airport +'/' + str(d.date()).replace('-','') + '.csv'
        print filepath
        df0.to_csv(filepath, index=False)



        t = 3
        time.sleep(t)

        if type(df0.dateutc[0]) == pd.tslib.Timestamp:
            continue
        else:
            print "Something is wrong"
            break

    print "Files for %s have been written" % airport
    return

In [84]:
write_daily_weather_data('kalb', dates)

../data/wunderground/kalb/20090921.csv
../data/wunderground/kalb/20090922.csv
../data/wunderground/kalb/20090923.csv
../data/wunderground/kalb/20090924.csv
../data/wunderground/kalb/20090925.csv
../data/wunderground/kalb/20090926.csv
../data/wunderground/kalb/20090927.csv
../data/wunderground/kalb/20090928.csv
../data/wunderground/kalb/20090929.csv
../data/wunderground/kalb/20090930.csv
../data/wunderground/kalb/20091001.csv
../data/wunderground/kalb/20091002.csv
../data/wunderground/kalb/20091003.csv
../data/wunderground/kalb/20091004.csv
../data/wunderground/kalb/20091005.csv
../data/wunderground/kalb/20091006.csv
../data/wunderground/kalb/20091007.csv
../data/wunderground/kalb/20091008.csv
../data/wunderground/kalb/20091009.csv
../data/wunderground/kalb/20091010.csv
../data/wunderground/kalb/20091011.csv
../data/wunderground/kalb/20091012.csv
../data/wunderground/kalb/20091013.csv
../data/wunderground/kalb/20091014.csv
../data/wunderground/kalb/20091015.csv
../data/wunderground/kalb

In [45]:
for a in airports:
    write_daily_weather_data(a, dates)

../data/wunderground/klga/20120101.csv
../data/wunderground/klga/20120102.csv
../data/wunderground/klga/20120103.csv
../data/wunderground/klga/20120104.csv
../data/wunderground/klga/20120105.csv
../data/wunderground/klga/20120106.csv
../data/wunderground/klga/20120107.csv
../data/wunderground/klga/20120108.csv
../data/wunderground/klga/20120109.csv
../data/wunderground/klga/20120110.csv
../data/wunderground/klga/20120111.csv
../data/wunderground/klga/20120112.csv
../data/wunderground/klga/20120113.csv
../data/wunderground/klga/20120114.csv
../data/wunderground/klga/20120115.csv
../data/wunderground/klga/20120116.csv
../data/wunderground/klga/20120117.csv
../data/wunderground/klga/20120118.csv
../data/wunderground/klga/20120119.csv
../data/wunderground/klga/20120120.csv
../data/wunderground/klga/20120121.csv
../data/wunderground/klga/20120122.csv
../data/wunderground/klga/20120123.csv
../data/wunderground/klga/20120124.csv
../data/wunderground/klga/20120125.csv
../data/wunderground/klga

URLError: <urlopen error [Errno 54] Connection reset by peer>

In [85]:

def combine_weather_data(airport):
    csvs = []
    for file in os.listdir("../data/wunderground/"+airport+"/"):
        if file.endswith(".csv"):
            csvs.append(file)

    fout=open("../data/wunderground/"+airport+"_all.csv","a")

    # write the entire first file:
    for line in open("../data/wunderground/"+airport+"/"+csvs[0]):
        fout.write(line)
    # now the rest, skipping the headers:    
    for file in csvs[1:]:
        f = open("../data/wunderground/"+airport+"/"+file)
        f.next() # skip the header
        for line in f:
             fout.write(line)
        f.close() # not really needed
    fout.close()
    print "Files for %s have been combined" % airport

In [86]:
combine_weather_data('kalb')

Files for kalb have been combined


In [87]:
a = pd.read_csv("../data/wunderground/kalb_all.csv")

In [88]:
len(a)

160253

In [93]:
len(dates)*24

130296